# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [48]:
# import libraries
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import pickle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score

from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package punkt to /Users/thomas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/thomas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/thomas/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/thomas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)  
X = df.message
Y = df.drop(columns=["id", "message", "original", "genre"])

In [16]:
category_names = list(df.columns[4:])

In [17]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [19]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [20]:
def tokenize(text):
    # Normalize
    text = text.lower().strip()
    text = re.sub(r"[^a-z0-9]", " ", text)
    # Tokenize
    token = word_tokenize(text)
    # Remove stop words
    filtered_words = [word for word in token if word not in stopwords.words('english')]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(word) for word in filtered_words]
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [21]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [23]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

In [24]:
y_pred = pipeline.predict(X_test)

In [26]:
y_pred_nice = pd.DataFrame (y_pred, columns = y_test.columns)
y_pred_nice

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6549,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6550,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6551,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
6552,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
labels = y_test.columns
labels

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [42]:
for i in range(len(category_names)):
    print('Category {}: {} '.format(i, category_names[i]))
    print(classification_report(y_test.iloc[:, i].values, y_pred[:, i]))
    print('Accuracy {}\n\n'.format(accuracy_score(y_test.iloc[:, i].values, y_pred[:, i])))
accuracy = (y_pred == y_test.values).mean()
print('The model accuracy is {:.3f}'.format(accuracy))

Category 0: related 
              precision    recall  f1-score   support

           0       0.73      0.43      0.54      1546
           1       0.84      0.95      0.89      4964
           2       0.62      0.18      0.28        44

    accuracy                           0.82      6554
   macro avg       0.73      0.52      0.57      6554
weighted avg       0.81      0.82      0.80      6554

Accuracy 0.8227036924015868


Category 1: request 
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5420
           1       0.82      0.47      0.60      1134

    accuracy                           0.89      6554
   macro avg       0.86      0.72      0.77      6554
weighted avg       0.88      0.89      0.88      6554

Accuracy 0.8901434238632896


Category 2: offer 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6533
           1       0.00      0.00      0.00        21

    acc

/Users/thomas/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6217
           1       0.81      0.09      0.16       337

    accuracy                           0.95      6554
   macro avg       0.88      0.54      0.57      6554
weighted avg       0.95      0.95      0.93      6554

Accuracy 0.9520903265181568


Category 6: search_and_rescue 
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      6374
           1       0.67      0.06      0.10       180

    accuracy                           0.97      6554
   macro avg       0.82      0.53      0.54      6554
weighted avg       0.97      0.97      0.96      6554

Accuracy 0.9732987488556607


Category 7: security 
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6421
           1       0.00      0.00      0.00       133

    accuracy                           0.98      6554
   macro avg   

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6273
           1       0.00      0.00      0.00       281

    accuracy                           0.96      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.92      0.96      0.94      6554

Accuracy 0.9571254195910894


Category 28: weather_related 
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      4708
           1       0.85      0.71      0.77      1846

    accuracy                           0.88      6554
   macro avg       0.87      0.83      0.85      6554
weighted avg       0.88      0.88      0.88      6554

Accuracy 0.882056759231004


Category 29: floods 
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5988
           1       0.90      0.45      0.60       566

    accuracy                           0.95      6554
   macro avg      

In [29]:
print(classification_report(y_test.iloc[:, 1:].values, np.array([x[1:] for x in y_pred])))#, target_names = category_names))

              precision    recall  f1-score   support

           0       0.82      0.47      0.60      1134
           1       0.00      0.00      0.00        21
           2       0.75      0.70      0.73      2713
           3       0.72      0.07      0.13       529
           4       0.81      0.09      0.16       337
           5       0.67      0.06      0.10       180
           6       0.00      0.00      0.00       133
           7       0.61      0.06      0.11       225
           8       0.00      0.00      0.00         0
           9       0.92      0.34      0.49       448
          10       0.85      0.60      0.70       747
          11       0.83      0.35      0.49       598
          12       1.00      0.10      0.18       102
          13       0.83      0.03      0.06       153
          14       1.00      0.01      0.03        68
          15       0.73      0.04      0.07       206
          16       0.79      0.13      0.22       294
          17       0.65    

/Users/thomas/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thomas/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thomas/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [31]:
parameters = pipeline.get_params()
parameters

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x7f9c73ce7dd0>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=No

In [40]:
parameters = {
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4]
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

In [41]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [43]:
print(classification_report(y_test.iloc[:, 1:].values, np.array([x[1:] for x in y_pred])))#, target_names = category_names))

              precision    recall  f1-score   support

           0       0.82      0.47      0.60      1134
           1       0.00      0.00      0.00        21
           2       0.75      0.70      0.73      2713
           3       0.72      0.07      0.13       529
           4       0.81      0.09      0.16       337
           5       0.67      0.06      0.10       180
           6       0.00      0.00      0.00       133
           7       0.61      0.06      0.11       225
           8       0.00      0.00      0.00         0
           9       0.92      0.34      0.49       448
          10       0.85      0.60      0.70       747
          11       0.83      0.35      0.49       598
          12       1.00      0.10      0.18       102
          13       0.83      0.03      0.06       153
          14       1.00      0.01      0.03        68
          15       0.73      0.04      0.07       206
          16       0.79      0.13      0.22       294
          17       0.65    

/Users/thomas/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thomas/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thomas/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
accuracy = (y_pred == y_test.values).mean()
print('The model accuracy is {:.3f}'.format(accuracy))

The model accuracy is 0.948


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [49]:
with open ("classifier.pkl", 'wb') as f:
    pickle.dump(cv, f)  

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.